# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [1]:
from io import BytesIO
from datetime import datetime
import csv
from pathlib import Path

import requests
import pdfplumber

In [2]:
url = "https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_12.1.2025.pdf"
req = requests.get(url)

In [3]:
pdf = pdfplumber.open(BytesIO(req.content))

In [6]:
# assumes a 2-page PDF with a table spanning both pages
assert len(pdf.pages) == 2

In [7]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [8]:
lines

[['Statewide Report by County - December 1, 2025',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'Inactive',
  'Total Active'],
 ['Aurora', '486', '2', '992', '229', '3', '97', '76', '1,809'],
 ['Beadle', '2,402', '30', '5,210', '1,355', '44', '1,221', '533', '10,262'],
 ['Bennett', '651', '8', '727', '193', '3', '198', '126', '1,780'],
 ['Bon Homme', '832', '10', '2,392', '480', '', '229', '256', '3,943'],
 ['Brookings',
  '5,059',
  '106',
  '10,191',
  '3,650',
  '22',
  '2,877',
  '1047',
  '21,905'],
 ['Brown', '6,309', '88', '12,241', '3,169', '10', '1,763', '2818', '23,580'],
 ['Brule', '844', '12', '1,730', '444', '4', '221', '187', '3,255'],
 ['Buffalo', '528', '2', '150', '78', '4', '100', '55', '862'],
 ['Butte', '732', '54', '4,609', '861', '22', '658', '995', '6,936'],
 ['Campbell', '95', '1', '887', '69', '', '49', '79', '1,101'],
 ['Charles Mix',

In [9]:
# report date is in the first line
report_date = lines.pop(0)

In [10]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2025-12-01


In [11]:
# save the PDF
outpath = Path('pdfs') / f'{report_date}.pdf'

with open(outpath, 'wb') as outfile:
    outfile.write(req.content)

print(f'Wrote {outpath}')

Wrote pdfs/2025-12-01.pdf


In [12]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [13]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [14]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'inactive',
 'total active']

In [15]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [17]:
# doublecheck: 66 counties in SD
assert len(data) == 66

In [18]:
data[0]

{'county': 'Aurora',
 'democratic': 486,
 'libertarian': 2,
 'republican': 992,
 'independent': 229,
 'other': 3,
 'npa': 97,
 'inactive': 76,
 'date': '2025-12-01'}

In [19]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [20]:
# write to file
filepath = (Path('data') / f'{report_date}.csv').resolve()

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: /Users/cjwinchester/south-dakota-voter-registration-data/data/2025-12-01.csv


In [21]:
%run build_files.py

Party list: americans_elect, constitution, democratic, inactive, independent, libertarian, no_labels, npa, npa_ind, npa_ind_oth, other, reform, republican
--------------------
Wrote sd-voter-registration-data.csv
Wrote sd-voter-registration-data-simplified.csv
Wrote README.md
